In [ ]:
import numpy as np
import pandas as pd 


In [ ]:
movies = pd.read_csv('/content/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/tmdb_5000_credits.csv')


In [ ]:
movies.head()

In [ ]:
credits.head()

In [ ]:
movies = movies.merge(credits,on='title')
movies = movies[['movie_id','title','release_date','vote_average','overview','genres','keywords','cast','crew']]
movies.head()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
import ast
def convert(obj):
    L= []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies.head()

In [ ]:
def convert3(obj):
    L = []
    counter = 0 
    for i in ast.literal_eval(obj):
        if counter != 3:
           L.append(i['name'])
           counter+=1
        else:
          break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head()

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
           L.append(i['name'])
           break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies.head()

In [ ]:
movies['tags'] = movies['overview']+ movies['genres']+movies['keywords']+movies['cast']+movies['crew']
new_movies = movies[['movie_id','title','release_date','vote_average','tags']]
new_movies

In [ ]:
new_movies['tags'] = new_movies['tags'].apply(lambda x:" ".join(x))
new_movies

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vectors = cv.fit_transform(new_movies['tags']).toarray()
vectors

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()



In [ ]:
def stem(text):
    y = []

    for i in text.split():
        y.append(ps.stem(i)) 
    return " ".join(y)

In [ ]:
new_movies['tags'] = new_movies['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)
similarity

In [ ]:
def recommend(movie):
    index = new_movies[new_movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_movies.iloc[i[0]].title)

        


In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle
pickle.dump(new_movies.to_dict(),open('movie_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))
